尝试用free的点做一次poi，可能精细一点
尝试过后poi没有面积，于是第二版本


最后分成9类：output_data/LAD_free_POI_group_with_9_category_summary_wide.csv


In [6]:

import geopandas as gpd
import pandas as pd


# 1. 读取两个 shp 文件
lad_path = 'POI/LAD_MAY_2021_UK_BUC.shp'
poi_path = 'POI/gis_osm_pois_free_1.shp'

lad_gdf = gpd.read_file(lad_path)
poi_gdf = gpd.read_file(poi_path)

# 2. 投影统一为英国国家网格坐标系 EPSG:27700
target_crs = 'EPSG:27700'
lad_gdf = lad_gdf.to_crs(target_crs)
poi_gdf = poi_gdf.to_crs(target_crs)

# 3. 空间连接，将每个POI归属到对应LAD
joined = gpd.sjoin(poi_gdf, lad_gdf[['LAD21CD', 'LAD21NM', 'geometry']], how='left', predicate='intersects')

# 4. 计算面积（如果POI是面要素）
if joined.geom_type.isin(['Polygon', 'MultiPolygon']).any():
    joined['area'] = joined.geometry.area  # 单位为平方米
else:
    joined['area'] = 0

# 5. 按 LAD 和 fclass 统计
grouped = (
    joined.groupby(['LAD21CD', 'LAD21NM', 'fclass'])
    .agg(
        count=('osm_id', 'count'),
        total_area=('area', 'sum')
    )
    .reset_index()
)

# 6. 计算每个 LAD 下所有 POI 的总面积
lad_area_sum = grouped.groupby('LAD21CD')['total_area'].sum().reset_index().rename(columns={'total_area': 'lad_total_area'})
grouped = grouped.merge(lad_area_sum, on='LAD21CD', how='left')

# 7. 计算面积比例
grouped['area_ratio'] = grouped['total_area'] / grouped['lad_total_area']

# 显示前几行
print(grouped.head())



     LAD21CD     LAD21NM          fclass  count  total_area  lad_total_area  \
0  E06000001  Hartlepool  archaeological      6           0               0   
1  E06000001  Hartlepool     arts_centre      1           0               0   
2  E06000001  Hartlepool         artwork     16           0               0   
3  E06000001  Hartlepool             atm     27           0               0   
4  E06000001  Hartlepool          bakery      8           0               0   

   area_ratio  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  


In [12]:
# 1. 空间连接，将每个POI归属到对应LAD
joined = gpd.sjoin(
    poi_gdf, 
    lad_gdf[['LAD21CD', 'LAD21NM', 'geometry']], 
    how='left', 
    predicate='intersects'
)

# 2. 按 LAD 和 fclass 分类统计数量
grouped = (
    joined.groupby(['LAD21CD', 'LAD21NM', 'fclass'])
    .agg(count=('osm_id', 'count'))
    .reset_index()
)
#  显示前几行
print(grouped.head())

     LAD21CD     LAD21NM          fclass  count
0  E06000001  Hartlepool  archaeological      6
1  E06000001  Hartlepool     arts_centre      1
2  E06000001  Hartlepool         artwork     16
3  E06000001  Hartlepool             atm     27
4  E06000001  Hartlepool          bakery      8


统计一下所有出现过的 fclass（兴趣点类型）有多少种，并列出它们。

In [13]:

df = grouped
fclass_list = sorted(df['fclass'].unique())

# 保存到 CSV 文件
# pd.DataFrame({'fclass': fclass_list}).to_csv('output_data/LAD_free_POI_group_fclass_list.csv', index=False, encoding='utf-8')

print(f"兴趣点类型总数：{len(fclass_list)}")


兴趣点类型总数：138


LAD_free_POI_group_fclass_list.csv

| category_code | category_name                | 说明                                 |
|---------------|-----------------------------|--------------------------------------|
| POI_RTL       | Retail                      | 零售                                 |
| POI_AF        | Accommodation & Food        | 住宿与餐饮                           |
| POI_AR        | Arts & Recreation           | 艺术与娱乐                           |
| POI_FI        | Financial Institutions      | 金融机构                             |
| POI_IC        | Information & Communication | 信息与通信                           |
| POI_PHS       | Personal & Household Svcs   | 个人与家庭服务                       |
| POI_HCC       | Healthcare Centers          | 医疗健康                             |
| POI_TF        | Transportation Facilities   | 交通设施                             |
| POI_OTR       | Other                       | 其他/未分类（补充，便于后续完善）    |


| fclass               | category_code | category_name                |
|----------------------|--------------|-----------------------------|
| alpine_hut           | POI_AF       | Accommodation & Food        |
| archaeological       | POI_AR       | Arts & Recreation           |
| arts_centre          | POI_AR       | Arts & Recreation           |
| artwork              | POI_AR       | Arts & Recreation           |
| atm                  | POI_FI       | Financial Institutions      |
| attraction           | POI_AR       | Arts & Recreation           |
| bakery               | POI_RTL      | Retail                      |
| bank                 | POI_FI       | Financial Institutions      |
| bar                  | POI_AF       | Accommodation & Food        |
| battlefield          | POI_AR       | Arts & Recreation           |
| beauty_shop          | POI_PHS      | Personal & Household Svcs   |
| bench                | POI_OTR      | Other                       |
| beverages            | POI_RTL      | Retail                      |
| bicycle_rental       | POI_TF       | Transportation Facilities   |
| bicycle_shop         | POI_RTL      | Retail                      |
| biergarten           | POI_AF       | Accommodation & Food        |
| bookshop             | POI_RTL      | Retail                      |
| butcher              | POI_RTL      | Retail                      |
| cafe                 | POI_AF       | Accommodation & Food        |
| camera_surveillance  | POI_OTR      | Other                       |
| camp_site            | POI_AF       | Accommodation & Food        |
| car_dealership       | POI_RTL      | Retail                      |
| car_rental           | POI_TF       | Transportation Facilities   |
| car_sharing          | POI_TF       | Transportation Facilities   |
| car_wash             | POI_PHS      | Personal & Household Svcs   |
| caravan_site         | POI_AF       | Accommodation & Food        |
| castle               | POI_AR       | Arts & Recreation           |
| chalet               | POI_AF       | Accommodation & Food        |
| chemist              | POI_HCC      | Healthcare Centers          |
| cinema               | POI_AR       | Arts & Recreation           |
| clinic               | POI_HCC      | Healthcare Centers          |
| clothes              | POI_RTL      | Retail                      |
| college              | POI_OTR      | Other                       |
| comms_tower          | POI_IC       | Information & Communication |
| community_centre     | POI_AR       | Arts & Recreation           |
| computer_shop        | POI_RTL      | Retail                      |
| convenience          | POI_RTL      | Retail                      |
| courthouse           | POI_OTR      | Other                       |
| dentist              | POI_HCC      | Healthcare Centers          |
| department_store     | POI_RTL      | Retail                      |
| doctors              | POI_HCC      | Healthcare Centers          |
| dog_park             | POI_AR       | Arts & Recreation           |
| doityourself         | POI_RTL      | Retail                      |
| drinking_water       | POI_OTR      | Other                       |
| embassy              | POI_OTR      | Other                       |
| fast_food            | POI_AF       | Accommodation & Food        |
| fire_station         | POI_OTR      | Other                       |
| florist              | POI_RTL      | Retail                      |
| food_court           | POI_AF       | Accommodation & Food        |
| fort                 | POI_AR       | Arts & Recreation           |
| fountain             | POI_AR       | Arts & Recreation           |
| furniture_shop       | POI_RTL      | Retail                      |
| garden_centre        | POI_RTL      | Retail                      |
| general              | POI_OTR      | Other                       |
| gift_shop            | POI_RTL      | Retail                      |
| golf_course          | POI_AR       | Arts & Recreation           |
| graveyard            | POI_AR       | Arts & Recreation           |
| greengrocer          | POI_RTL      | Retail                      |
| guesthouse           | POI_AF       | Accommodation & Food        |
| hairdresser          | POI_PHS      | Personal & Household Svcs   |
| hospital             | POI_HCC      | Healthcare Centers          |
| hostel               | POI_AF       | Accommodation & Food        |
| hotel                | POI_AF       | Accommodation & Food        |
| hunting_stand        | POI_OTR      | Other                       |
| ice_rink             | POI_AR       | Arts & Recreation           |
| jeweller             | POI_RTL      | Retail                      |
| kindergarten         | POI_OTR      | Other                       |
| kiosk                | POI_RTL      | Retail                      |
| laundry              | POI_PHS      | Personal & Household Svcs   |
| library              | POI_AR       | Arts & Recreation           |
| lighthouse           | POI_AR       | Arts & Recreation           |
| mall                 | POI_RTL      | Retail                      |
| market_place         | POI_RTL      | Retail                      |
| memorial             | POI_AR       | Arts & Recreation           |
| mobile_phone_shop    | POI_RTL      | Retail                      |
| monument             | POI_AR       | Arts & Recreation           |
| motel                | POI_AF       | Accommodation & Food        |
| museum               | POI_AR       | Arts & Recreation           |
| newsagent            | POI_RTL      | Retail                      |
| nightclub            | POI_AR       | Arts & Recreation           |
| nursing_home         | POI_HCC      | Healthcare Centers          |
| observation_tower    | POI_AR       | Arts & Recreation           |
| optician             | POI_HCC      | Healthcare Centers          |
| outdoor_shop         | POI_RTL      | Retail                      |
| park                 | POI_AR       | Arts & Recreation           |
| pharmacy             | POI_HCC      | Healthcare Centers          |
| picnic_site          | POI_AR       | Arts & Recreation           |
| pitch                | POI_AR       | Arts & Recreation           |
| playground           | POI_AR       | Arts & Recreation           |
| police               | POI_OTR      | Other                       |
| post_box             | POI_IC       | Information & Communication |
| post_office          | POI_IC       | Information & Communication |
| prison               | POI_OTR      | Other                       |
| pub                  | POI_AF       | Accommodation & Food        |
| public_building      | POI_OTR      | Other                       |
| recycling            | POI_OTR      | Other                       |
| recycling_clothes    | POI_OTR      | Other                       |
| recycling_glass      | POI_OTR      | Other                       |
| recycling_metal      | POI_OTR      | Other                       |
| recycling_paper      | POI_OTR      | Other                       |
| restaurant           | POI_AF       | Accommodation & Food        |
| ruins                | POI_AR       | Arts & Recreation           |
| school               | POI_OTR      | Other                       |
| shelter              | POI_OTR      | Other                       |
| shoe_shop            | POI_RTL      | Retail                      |
| sports_centre        | POI_AR       | Arts & Recreation           |
| sports_shop          | POI_RTL      | Retail                      |
| stadium              | POI_AR       | Arts & Recreation           |
| stationery           | POI_RTL      | Retail                      |
| supermarket          | POI_RTL      | Retail                      |
| swimming_pool        | POI_AR       | Arts & Recreation           |
| telephone            | POI_IC       | Information & Communication |
| theatre              | POI_AR       | Arts & Recreation           |
| theme_park           | POI_AR       | Arts & Recreation           |
| toilet               | POI_OTR      | Other                       |
| tourist_info         | POI_IC       | Information & Communication |
| tower                | POI_AR       | Arts & Recreation           |
| town_hall            | POI_OTR      | Other                       |
| toy_shop             | POI_RTL      | Retail                      |
| track                | POI_AR       | Arts & Recreation           |
| travel_agent         | POI_RTL      | Retail                      |
| university           | POI_OTR      | Other                       |
| vending_any          | POI_RTL      | Retail                      |
| vending_machine      | POI_RTL      | Retail                      |
| vending_parking      | POI_TF       | Transportation Facilities   |
| veterinary           | POI_HCC      | Healthcare Centers          |
| video_shop           | POI_RTL      | Retail                      |
| viewpoint            | POI_AR       | Arts & Recreation           |
| waste_basket         | POI_OTR      | Other                       |
| wastewater_plant     | POI_OTR      | Other                       |
| water_mill           | POI_AR       | Arts & Recreation           |
| water_tower          | POI_OTR      | Other                       |
| water_well           | POI_OTR      | Other                       |
| water_works          | POI_OTR      | Other                       |
| wayside_cross        | POI_AR       | Arts & Recreation           |
| wayside_shrine       | POI_AR       | Arts & Recreation           |
| windmill             | POI_AR       | Arts & Recreation           |
| zoo                  | POI_AR       | Arts & Recreation           |


In [14]:



# 1. 构建 fclass 到九大类的映射字典
fclass_to_category = {
    'alpine_hut': 'POI_AF',
    'archaeological': 'POI_AR',
    'arts_centre': 'POI_AR',
    'artwork': 'POI_AR',
    'atm': 'POI_FI',
    'attraction': 'POI_AR',
    'bakery': 'POI_RTL',
    'bank': 'POI_FI',
    'bar': 'POI_AF',
    'battlefield': 'POI_AR',
    'beauty_shop': 'POI_PHS',
    'bench': 'POI_OTR',
    'beverages': 'POI_RTL',
    'bicycle_rental': 'POI_TF',
    'bicycle_shop': 'POI_RTL',
    'biergarten': 'POI_AF',
    'bookshop': 'POI_RTL',
    'butcher': 'POI_RTL',
    'cafe': 'POI_AF',
    'camera_surveillance': 'POI_OTR',
    'camp_site': 'POI_AF',
    'car_dealership': 'POI_RTL',
    'car_rental': 'POI_TF',
    'car_sharing': 'POI_TF',
    'car_wash': 'POI_PHS',
    'caravan_site': 'POI_AF',
    'castle': 'POI_AR',
    'chalet': 'POI_AF',
    'chemist': 'POI_HCC',
    'cinema': 'POI_AR',
    'clinic': 'POI_HCC',
    'clothes': 'POI_RTL',
    'college': 'POI_OTR',
    'comms_tower': 'POI_IC',
    'community_centre': 'POI_AR',
    'computer_shop': 'POI_RTL',
    'convenience': 'POI_RTL',
    'courthouse': 'POI_OTR',
    'dentist': 'POI_HCC',
    'department_store': 'POI_RTL',
    'doctors': 'POI_HCC',
    'dog_park': 'POI_AR',
    'doityourself': 'POI_RTL',
    'drinking_water': 'POI_OTR',
    'embassy': 'POI_OTR',
    'fast_food': 'POI_AF',
    'fire_station': 'POI_OTR',
    'florist': 'POI_RTL',
    'food_court': 'POI_AF',
    'fort': 'POI_AR',
    'fountain': 'POI_AR',
    'furniture_shop': 'POI_RTL',
    'garden_centre': 'POI_RTL',
    'general': 'POI_OTR',
    'gift_shop': 'POI_RTL',
    'golf_course': 'POI_AR',
    'graveyard': 'POI_AR',
    'greengrocer': 'POI_RTL',
    'guesthouse': 'POI_AF',
    'hairdresser': 'POI_PHS',
    'hospital': 'POI_HCC',
    'hostel': 'POI_AF',
    'hotel': 'POI_AF',
    'hunting_stand': 'POI_OTR',
    'ice_rink': 'POI_AR',
    'jeweller': 'POI_RTL',
    'kindergarten': 'POI_OTR',
    'kiosk': 'POI_RTL',
    'laundry': 'POI_PHS',
    'library': 'POI_AR',
    'lighthouse': 'POI_AR',
    'mall': 'POI_RTL',
    'market_place': 'POI_RTL',
    'memorial': 'POI_AR',
    'mobile_phone_shop': 'POI_RTL',
    'monument': 'POI_AR',
    'motel': 'POI_AF',
    'museum': 'POI_AR',
    'newsagent': 'POI_RTL',
    'nightclub': 'POI_AR',
    'nursing_home': 'POI_HCC',
    'observation_tower': 'POI_AR',
    'optician': 'POI_HCC',
    'outdoor_shop': 'POI_RTL',
    'park': 'POI_AR',
    'pharmacy': 'POI_HCC',
    'picnic_site': 'POI_AR',
    'pitch': 'POI_AR',
    'playground': 'POI_AR',
    'police': 'POI_OTR',
    'post_box': 'POI_IC',
    'post_office': 'POI_IC',
    'prison': 'POI_OTR',
    'pub': 'POI_AF',
    'public_building': 'POI_OTR',
    'recycling': 'POI_OTR',
    'recycling_clothes': 'POI_OTR',
    'recycling_glass': 'POI_OTR',
    'recycling_metal': 'POI_OTR',
    'recycling_paper': 'POI_OTR',
    'restaurant': 'POI_AF',
    'ruins': 'POI_AR',
    'school': 'POI_OTR',
    'shelter': 'POI_OTR',
    'shoe_shop': 'POI_RTL',
    'sports_centre': 'POI_AR',
    'sports_shop': 'POI_RTL',
    'stadium': 'POI_AR',
    'stationery': 'POI_RTL',
    'supermarket': 'POI_RTL',
    'swimming_pool': 'POI_AR',
    'telephone': 'POI_IC',
    'theatre': 'POI_AR',
    'theme_park': 'POI_AR',
    'toilet': 'POI_OTR',
    'tourist_info': 'POI_IC',
    'tower': 'POI_AR',
    'town_hall': 'POI_OTR',
    'toy_shop': 'POI_RTL',
    'track': 'POI_AR',
    'travel_agent': 'POI_RTL',
    'university': 'POI_OTR',
    'vending_any': 'POI_RTL',
    'vending_machine': 'POI_RTL',
    'vending_parking': 'POI_TF',
    'veterinary': 'POI_HCC',
    'video_shop': 'POI_RTL',
    'viewpoint': 'POI_AR',
    'waste_basket': 'POI_OTR',
    'wastewater_plant': 'POI_OTR',
    'water_mill': 'POI_AR',
    'water_tower': 'POI_OTR',
    'water_well': 'POI_OTR',
    'water_works': 'POI_OTR',
    'wayside_cross': 'POI_AR',
    'wayside_shrine': 'POI_AR',
    'windmill': 'POI_AR',
    'zoo': 'POI_AR'
}

# 2. 添加分类字段
df['category_code'] = df['fclass'].map(fclass_to_category)

# 3. 添加分类名称字段
category_code_to_name = {
    'POI_RTL': 'Retail',
    'POI_AF': 'Accommodation_and_Food',
    'POI_AR': 'Arts_and_Recreation',
    'POI_FI': 'Financial_Institutions',
    'POI_IC': 'Information_and_Communication',
    'POI_PHS': 'Personal_and_Household_Svcs',
    'POI_HCC': 'Healthcare_Centers',
    'POI_TF': 'Transportation_Facilities',
    'POI_OTR': 'Other'
}
df['category_name'] = df['category_code'].map(category_code_to_name)



输出每个区域每个类别的POI数量。
输出说明
LAD_POI_category_summary_long.csv：每行是一个区域-类别，包含数量，适合进一步分析或可视化。
LAD_POI_category_summary_wide.csv：每行是一个区域，每列是一个类别，数值为POI数量，适合做区域对比。

In [15]:

# 按区域和九大类汇总统计
summary = df.groupby(['LAD21CD', 'LAD21NM', 'category_code', 'category_name'])['count'].sum().reset_index()

# 将九大类展开为列，区域为行
pivot = summary.pivot_table(index=['LAD21CD', 'LAD21NM'],
                            columns='category_name',
                            values='count',
                            fill_value=0).reset_index()

# 保存汇总结果
# summary.to_csv('output_data/LAD_free_POI_group_with_9_category_summary_long.csv', index=False)  # 长表格式
pivot.to_csv('output_data/LAD_free_POI_group_with_9_category_summary_wide.csv', index=False)    # 宽表格式
